# Image Classification with SVM, MLP, KNN

## SVM

### Reference: https://github.com/whimian/SVM-Image-Classification

In [22]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [23]:
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
%matplotlib notebook
from sklearn import svm, metrics, datasets
from sklearn.utils import Bunch
from sklearn.model_selection import GridSearchCV, train_test_split

from skimage.io import imread
from skimage.transform import resize

### Load images in structured directory like it's sklearn sample dataset

In [24]:
def load_image_files(container_path, dimension=(64, 64)):
    """
    Load image files with categories as subfolder names 
    which performs like scikit-learn sample dataset
    
    Parameters
    ----------
    container_path : string or unicode
        Path to the main folder holding one subfolder per category
    dimension : tuple
        size to which image are adjusted to
        
    Returns
    -------
    Bunch
    """
    image_dir = Path(container_path)
    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    categories = [fo.name for fo in folders]

    descr = "A image classification dataset"
    images = []
    flat_data = []
    target = []
    for i, direc in enumerate(folders):
        for file in direc.iterdir():
            img = imread(file)
            img_resized = resize(img, dimension, anti_aliasing=True, mode='reflect')
            flat_data.append(img_resized.flatten()) 
            images.append(img_resized)
            target.append(i)
    flat_data = np.array(flat_data)
    target = np.array(target)
    images = np.array(images)

    return Bunch(data=flat_data,
                 target=target,
                 target_names=categories,
                 images=images,
                 DESCR=descr)

In [25]:
image_dataset = load_image_files("dataset/")

### Split data

In [26]:
X_train, X_test, y_train, y_test = train_test_split(
    image_dataset.data, image_dataset.target, test_size=0.3,random_state=109)

### Train data with parameter optimization

In [27]:
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]
svc = svm.SVC()
clf = GridSearchCV(svc, param_grid)
clf.fit(X_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']}, {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [33]:
# Best paramete set
print('Best parameters found:\n', clf.best_params_)
print('\n')

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

Best parameters found:
 {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}


0.586 (+/-0.014) for {'C': 1, 'kernel': 'linear'}
0.586 (+/-0.014) for {'C': 10, 'kernel': 'linear'}
0.586 (+/-0.014) for {'C': 100, 'kernel': 'linear'}
0.586 (+/-0.014) for {'C': 1000, 'kernel': 'linear'}
0.679 (+/-0.051) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.627 (+/-0.057) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.673 (+/-0.025) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.658 (+/-0.059) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.666 (+/-0.046) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.629 (+/-0.008) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.666 (+/-0.046) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.623 (+/-0.004) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}


In [34]:
print('best score: %0.3f' %clf.best_score_)

best score: 0.679


### Predict

In [35]:
y_pred = clf.predict(X_test)

### Report

In [37]:
print("Classification report for - \n{}:\n{}\n".format(
    clf, metrics.classification_report(y_test, y_pred)))

Classification report for - 
GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']}, {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0):
              precision    recall  f1-score   support

           0       0.68      0.78      0.73       161
           1       0.69      0.57      0.63       138

   micro avg       0.69      0.69      0.69       299
   macro avg       0.69      0.68      0.68       299
weighted avg       0.69      0.69      0.68       299




In [68]:
# 預測成功的比例
print('訓練集: ',clf.score(X_train,y_train))
print('測試集: ',clf.score(X_test,y_test))

訓練集:  0.8949640287769784
測試集:  0.6387959866220736


## MLP

### Reference: https://www.kaggle.com/pandaqc/mlp-with-sickitlearn

In [1]:
# imports

# data analysis
import pandas as pd
import numpy as np
from scipy import stats, integrate

# machine learning
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import Bunch
from skimage.io import imread
from skimage.transform import resize

# data visualization
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from pathlib import Path
%matplotlib notebook

In [2]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [3]:
def load_image_files(container_path, dimension=(64, 64)):
    image_dir = Path(container_path)
    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    categories = [fo.name for fo in folders]

    descr = "A image classification dataset"
    images = []
    flat_data = []
    target = []
    for i, direc in enumerate(folders):
        for file in direc.iterdir():
            img = imread(file)
            img_resized = resize(img, dimension, anti_aliasing=True, mode='reflect')
            flat_data.append(img_resized.flatten()) 
            images.append(img_resized)
            target.append(i)
    flat_data = np.array(flat_data)
    target = np.array(target)
    images = np.array(images)

    return Bunch(data=flat_data,
                 target=target,
                 target_names=categories,
                 images=images,
                 DESCR=descr)

In [4]:
image_dataset = load_image_files("dataset/")

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    image_dataset.data, image_dataset.target, test_size=0.3,random_state=109)

In [6]:
print('X_train shape:', X_train.shape, 'y_train shape:', y_train.shape)
print('X_val shape:', X_test.shape, 'y_val shape:', y_test.shape)

X_train shape: (695, 12288) y_train shape: (695,)
X_val shape: (299, 12288) y_val shape: (299,)


In [38]:
# instanciate the MLP classifier
mlp = MLPClassifier(max_iter=100,verbose=True)

parameter_space = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam','lbfgs'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)

# fit the model with the training set
clf.fit(X_train, y_train)

Iteration 1, loss = 0.68151234
Iteration 2, loss = 0.66159792
Iteration 3, loss = 0.64534279
Iteration 4, loss = 0.63617046
Iteration 5, loss = 0.63410727
Iteration 6, loss = 0.62022041
Iteration 7, loss = 0.60828458
Iteration 8, loss = 0.60651048
Iteration 9, loss = 0.59605962
Iteration 10, loss = 0.59096369
Iteration 11, loss = 0.58464550
Iteration 12, loss = 0.57789966
Iteration 13, loss = 0.57607201
Iteration 14, loss = 0.56831476
Iteration 15, loss = 0.56305529
Iteration 16, loss = 0.55951831
Iteration 17, loss = 0.55531892
Iteration 18, loss = 0.55625251
Iteration 19, loss = 0.54843815
Iteration 20, loss = 0.54611627
Iteration 21, loss = 0.54189821
Iteration 22, loss = 0.53590090
Iteration 23, loss = 0.53557412
Iteration 24, loss = 0.52942558
Iteration 25, loss = 0.52738052
Iteration 26, loss = 0.52361644
Iteration 27, loss = 0.51902282
Iteration 28, loss = 0.51682205
Iteration 29, loss = 0.51497357
Iteration 30, loss = 0.51422712
Iteration 31, loss = 0.50749234
Iteration 32, los

/opt/conda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=100, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=True, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'hidden_layer_sizes': [(50, 50, 50), (50, 100, 50), (100,)], 'activation': ['tanh', 'relu'], 'solver': ['sgd', 'adam', 'lbfgs'], 'alpha': [0.0001, 0.05], 'learning_rate': ['constant', 'adaptive']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [41]:
# Best paramete set
print('Best parameters found:\n', clf.best_params_)
print('\n')

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

Best parameters found:
 {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant', 'solver': 'sgd'}


0.623 (+/-0.045) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'constant', 'solver': 'sgd'}
0.600 (+/-0.044) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'constant', 'solver': 'adam'}
0.626 (+/-0.021) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'constant', 'solver': 'lbfgs'}
0.633 (+/-0.055) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'adaptive', 'solver': 'sgd'}
0.588 (+/-0.040) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'adaptive', 'solver': 'adam'}
0.619 (+/-0.031) for {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'adaptive', 'solver': 'lb

In [42]:
# run prediction on the validation set 
y_pred = clf.predict(X_test)

print('Classification report:\n\n', classification_report(y_test, y_pred), '\n')
print('------------------------------\n')
print('Confusion matrix:\n\n', confusion_matrix(y_test, y_pred))

Classification report:

               precision    recall  f1-score   support

           0       0.64      0.76      0.69       161
           1       0.64      0.50      0.56       138

   micro avg       0.64      0.64      0.64       299
   macro avg       0.64      0.63      0.63       299
weighted avg       0.64      0.64      0.63       299
 

------------------------------

Confusion matrix:

 [[122  39]
 [ 69  69]]


In [46]:
# instanciate the MLP classifier
mlp = MLPClassifier(max_iter=100,verbose=True,activation="relu",alpha=0.0001,hidden_layer_sizes=(100,),learning_rate="constant",solver="sgd")

parameter_space =  {}

# fit the model with the training set
mlp.fit(X_train, y_train)

Iteration 1, loss = 0.81959653
Iteration 2, loss = 0.69784681
Iteration 3, loss = 0.66083969
Iteration 4, loss = 0.66063183
Iteration 5, loss = 0.65095846
Iteration 6, loss = 0.64445064
Iteration 7, loss = 0.63962452
Iteration 8, loss = 0.63597922
Iteration 9, loss = 0.63081218
Iteration 10, loss = 0.62609308
Iteration 11, loss = 0.62341456
Iteration 12, loss = 0.62094705
Iteration 13, loss = 0.61544579
Iteration 14, loss = 0.61348408
Iteration 15, loss = 0.60921989
Iteration 16, loss = 0.60627732
Iteration 17, loss = 0.60332135
Iteration 18, loss = 0.60182047
Iteration 19, loss = 0.59705204
Iteration 20, loss = 0.59474966
Iteration 21, loss = 0.59114742
Iteration 22, loss = 0.58832937
Iteration 23, loss = 0.58528373
Iteration 24, loss = 0.58290026
Iteration 25, loss = 0.58033197
Iteration 26, loss = 0.57792533
Iteration 27, loss = 0.57494881
Iteration 28, loss = 0.57268789
Iteration 29, loss = 0.56914969
Iteration 30, loss = 0.56704637
Iteration 31, loss = 0.56452684
Iteration 32, los

/opt/conda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=100, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='sgd', tol=0.0001,
       validation_fraction=0.1, verbose=True, warm_start=False)

In [67]:
# 預測成功的比例
print('訓練集: ',mlp.score(X_train,y_train))
print('測試集: ',mlp.score(X_test,y_test))

訓練集:  0.8460431654676259
測試集:  0.6287625418060201


## KNN

In [64]:
X_train, X_test, y_train, y_test = train_test_split(
    image_dataset.data, image_dataset.target, test_size=0.3,random_state=109)

In [79]:
from sklearn.neighbors import KNeighborsClassifier

# 建立KNN模型
knnModel = KNeighborsClassifier(n_jobs=-1)

k_range = list(range(1,10))
weight_options = ['uniform','distance']

param_gridknn = dict(n_neighbors = k_range,
                     weights = weight_options)

gridKNN = GridSearchCV(knnModel,
                       param_gridknn,
                       cv=5,
                       scoring='accuracy',
                       verbose=1,
                      n_jobs=-1)

gridKNN.fit(X_train,y_train)
print('best score is:',str(gridKNN.best_score_))
print('best params are:',str(gridKNN.best_params_))

Fitting 5 folds for each of 18 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   12.1s


best score is: 0.6532374100719425
best params are: {'n_neighbors': 2, 'weights': 'uniform'}


[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  1.7min finished


In [81]:
# 預測成功的比例
print('訓練集: ',gridKNN.score(X_train,y_train))
print('測試集: ',gridKNN.score(X_test,y_test))

訓練集:  0.8863309352517985
測試集:  0.6555183946488294


In [82]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.64      0.76      0.69       161
           1       0.64      0.50      0.56       138

   micro avg       0.64      0.64      0.64       299
   macro avg       0.64      0.63      0.63       299
weighted avg       0.64      0.64      0.63       299

